# Pre processing

### Import libaries

In [1]:
import os
import time
import xmltodict
import pandas as pd
import numpy as np
from azure.iot.device import IoTHubDeviceClient, Message
from datetime import datetime, timezone

#### Get path for all xml and txt files

In [2]:
rootdir = r"C:\Users\gisle\OneDrive - Equinor\Python\Datasets\WITSML Realtime drilling data"
extensions = ('.xml')
filelist=[]
subdirlist=[]

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        ext = os.path.splitext(file)[-1].lower()
        if ext in extensions:         
            try:
                subdirlist.append(((subdir).split('\\')[7]))
                filelist.append(os.path.join(subdir, file)) 
            except:
                pass
xmldict={}
wellborefolderlist=[]
for file in filelist:
    wellborefoldername=file.split('\\')[7]
    if wellborefoldername in wellborefolderlist:
        xmldict[wellborefoldername].append(file)
    else:
        wellborefolderlist.append(wellborefoldername)
        xmldict[wellborefoldername]=[file]

In [3]:
xmldict.keys()

dict_keys(['NA-NA-15_$47$_9-F-5', 'Norway-NA-15_$47$_9-F-1', 'Norway-NA-15_$47$_9-F-1 C', 'Norway-NA-15_$47$_9-F-11 B', 'Norway-NA-15_$47$_9-F-9 A', 'Norway-Statoil-15_$47$_9-F-12', 'Norway-Statoil-15_$47$_9-F-7', 'Norway-Statoil-NO 15_$47$_9-F-1 B', 'Norway-Statoil-NO 15_$47$_9-F-1 C', 'Norway-Statoil-NO 15_$47$_9-F-11', 'Norway-Statoil-NO 15_$47$_9-F-12', 'Norway-Statoil-NO 15_$47$_9-F-14', 'Norway-Statoil-NO 15_$47$_9-F-15', 'Norway-Statoil-NO 15_$47$_9-F-4', 'Norway-Statoil-NO 15_$47$_9-F-5', 'Norway-Statoil-NO 15_$47$_9-F-7', 'Norway-Statoil-NO 15_$47$_9-F-9', 'Norway-StatoilHydro-15_$47$_9-F-10', 'Norway-StatoilHydro-15_$47$_9-F-14', 'Norway-StatoilHydro-15_$47$_9-F-15', 'Norway-StatoilHydro-15_$47$_9-F-15A', 'Norway-StatoilHydro-15_$47$_9-F-15B', 'Norway-StatoilHydro-15_$47$_9-F-15S', 'Norway-StatoilHydro-15_$47$_9-F-4', 'Norway-StatoilHydro-15_$47$_9-F-5', 'Norway-StatoilHydro-15_$47$_9-F-9'])

In [16]:
list_of_paths=xmldict['Norway-StatoilHydro-15_$47$_9-F-15']

#### Load, parse XML Data and check for time logs

In [17]:
list_of_time_path=[]
dict_of_df={}
count=1
for path in list_of_paths:
    f = open(path, "r")
    xml=xmltodict.parse(f.read())
    f.close()
    try:
        mnemonic=xml['logs']['log']['logData']['mnemonicList'].split(',')
        if 'TIME' in mnemonic:
            data=[row.split(',') for row in xml['logs']['log']['logData']['data']]
            df=pd.DataFrame.from_records(data, columns=mnemonic).replace('', np.NaN)
            dict_of_df[str(count)]=df
            count+=1
    except Exception as exc:
        print(exc)

'logs'
'logs'
'logs'
'logs'


##### Merge all DFs to one large DF 

In [18]:
df_rop=pd.DataFrame(columns=['TIME', 'GS_ROP'])

In [19]:
df_rop=pd.DataFrame(columns=['TIME', 'GS_ROP'])
for df in dict_of_df:
    if 'GS_ROP' in dict_of_df[df].columns:
        df_rop=df_rop.append(dict_of_df[df][['TIME', 'GS_ROP']], ignore_index=True)

In [20]:
df_rop=df_rop.dropna()
df_rop['GS_ROP']=df_rop['GS_ROP'].astype('float').round(4)

In [21]:
df_rop.head()

,TIME,GS_ROP
1,2008-12-08T18:20:33.043Z,115.99
2,2008-12-08T18:20:37.968Z,115.99
3,2008-12-08T18:20:47.040Z,115.99
4,2008-12-08T18:20:51.014Z,115.99
5,2008-12-08T18:20:56.026Z,115.99


### Perform dump to Azure IoT hub

In [22]:
MSG_TXT = '{{"time": "{timestamp}","{value_name}": {value_py}}}'
value_name='ROPA'

In [23]:
last_value=0
counter=0
list_of_events=[]
for row in df_rop.iterrows():
    value_py=str(row[1]['GS_ROP'])
    timestamp=row[1]['TIME']
    if value_py!=last_value:
        msg_txt_formatted = MSG_TXT.format(timestamp=timestamp, value_name=value_name, value_py=value_py)
        list_of_events.append(msg_txt_formatted)
    last_value=value_py

In [24]:
list_of_events

['{"time": "2008-12-08T18:20:33.043Z","ROPA": 115.99}',
 '{"time": "2008-12-11T04:51:22.982Z","ROPA": 8.02}',
 '{"time": "2008-12-11T05:00:00.000Z","ROPA": 3.36}',
 '{"time": "2008-12-11T05:01:30.029Z","ROPA": 19.97}',
 '{"time": "2008-12-11T05:02:52.973Z","ROPA": 20.86}',
 '{"time": "2008-12-11T05:04:26.026Z","ROPA": 20.31}',
 '{"time": "2008-12-11T05:05:55.968Z","ROPA": 20.19}',
 '{"time": "2008-12-11T05:07:20.986Z","ROPA": 20.16}',
 '{"time": "2008-12-11T05:08:51.014Z","ROPA": 20.18}',
 '{"time": "2008-12-11T05:10:20.957Z","ROPA": 20.02}',
 '{"time": "2008-12-11T05:11:50.986Z","ROPA": 20.06}',
 '{"time": "2008-12-11T05:13:21.014Z","ROPA": 20.08}',
 '{"time": "2008-12-11T05:47:50.986Z","ROPA": 0.9}',
 '{"time": "2008-12-11T05:49:21.014Z","ROPA": 18.99}',
 '{"time": "2008-12-11T05:50:55.968Z","ROPA": 19.34}',
 '{"time": "2008-12-11T05:52:25.997Z","ROPA": 19.25}',
 '{"time": "2008-12-11T05:54:00.000Z","ROPA": 19.03}',
 '{"time": "2008-12-11T05:55:35.040Z","ROPA": 19.6}',
 '{"time": "20

##### Message format template

In [28]:
help(message)

Help on Message in module azure.iot.device.iothub.models.message object:

class Message(builtins.object)
 |  Message(data, message_id=None, content_encoding=None, content_type=None, output_name=None)
 |  
 |  Represents a message to or from IoTHub
 |  
 |  :ivar data: The data that constitutes the payload
 |  :ivar custom_properties: Dictionary of custom message properties. The keys and values of these properties will always be string.
 |  :ivar message id: A user-settable identifier for the message used for request-reply patterns. Format: A case-sensitive string (up to 128 characters long) of ASCII 7-bit alphanumeric characters + {'-', ':', '.', '+', '%', '_', '#', '*', '?', '!', '(', ')', ',', '=', '@', ';', '$', '''}
 |  :ivar expiry_time_utc: Date and time of message expiration in UTC format
 |  :ivar correlation_id: A property in a response message that typically contains the message_id of the request, in request-reply patterns
 |  :ivar user_id: An ID to specify the origin of mes

In [25]:
client = IoTHubDeviceClient.create_from_connection_string('ENTER CONNECTION STRING')

In [26]:
for event in list_of_events[:50]:
    time.sleep(0.1)
    print(event)
    message = Message(event)
    client.send_message(message)

{"time": "2008-12-08T18:20:33.043Z","ROPA": 115.99}
{"time": "2008-12-11T04:51:22.982Z","ROPA": 8.02}
{"time": "2008-12-11T05:00:00.000Z","ROPA": 3.36}
{"time": "2008-12-11T05:01:30.029Z","ROPA": 19.97}
{"time": "2008-12-11T05:02:52.973Z","ROPA": 20.86}
{"time": "2008-12-11T05:04:26.026Z","ROPA": 20.31}
{"time": "2008-12-11T05:05:55.968Z","ROPA": 20.19}
{"time": "2008-12-11T05:07:20.986Z","ROPA": 20.16}
{"time": "2008-12-11T05:08:51.014Z","ROPA": 20.18}
{"time": "2008-12-11T05:10:20.957Z","ROPA": 20.02}
{"time": "2008-12-11T05:11:50.986Z","ROPA": 20.06}


failed to receive on socket: [WinError 10054] An existing connection was forcibly closed by the remote host


{"time": "2008-12-11T05:13:21.014Z","ROPA": 20.08}
{"time": "2008-12-11T05:47:50.986Z","ROPA": 0.9}
{"time": "2008-12-11T05:49:21.014Z","ROPA": 18.99}
{"time": "2008-12-11T05:50:55.968Z","ROPA": 19.34}
{"time": "2008-12-11T05:52:25.997Z","ROPA": 19.25}
{"time": "2008-12-11T05:54:00.000Z","ROPA": 19.03}
{"time": "2008-12-11T05:55:35.040Z","ROPA": 19.6}
{"time": "2008-12-11T05:57:09.043Z","ROPA": 18.8}
{"time": "2008-12-11T05:58:43.997Z","ROPA": 19.78}
{"time": "2008-12-11T06:00:14.026Z","ROPA": 18.87}
{"time": "2008-12-11T06:01:48.029Z","ROPA": 19.21}
{"time": "2008-12-11T06:03:17.971Z","ROPA": 19.66}
{"time": "2008-12-11T06:04:53.011Z","ROPA": 19.13}
{"time": "2008-12-11T06:06:32.026Z","ROPA": 19.25}
{"time": "2008-12-11T06:08:01.968Z","ROPA": 19.44}
{"time": "2008-12-11T06:09:31.997Z","ROPA": 20.56}
{"time": "2008-12-11T06:10:43.968Z","ROPA": 24.68}
{"time": "2008-12-11T06:11:56.026Z","ROPA": 24.36}
{"time": "2008-12-11T06:13:11.971Z","ROPA": 24.05}
{"time": "2008-12-11T06:14:24.029Z"